<a href="https://colab.research.google.com/github/harryfb/Batch_Image_Classification/blob/master/advanced_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Jul  9 14:15:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8MB 26kB/s 
     |████████████████████████████████| 6.6MB 2.2MB/s 
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101
  Found existing installation: torchvision 0.6.1+cu101
    Uninstalling torchvision-0.6.1+cu101:
      Successfully uninstalled torchvision-0.6.1+cu101
1.5.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
     |████████████████████████████████| 6.2MB 996kB/s 
  Created wheel for fvcore: filename=fvcore-0.1.1.post20200704-cp36-none-any.whl size=41894 sha256=4c6231398f15510c21d762609b9499c12bdb46601baa84e94aa2da36c049f992
  Stored in directory: /root/.cache/pip/wheels/fb/d2/8e/b6d0f19811e77dabff1ebed6605ce2b59ee9f487079b434c8c
Successfully built fvcore


In [ ]:
!pip install wandb -q

     |████████████████████████████████| 1.4MB 2.8MB/s 
     |████████████████████████████████| 112kB 18.6MB/s 
     |████████████████████████████████| 102kB 7.9MB/s 
     |████████████████████████████████| 460kB 18.5MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 71kB 6.3MB/s 


In [ ]:
import wandb
import os
os.environ['WANDB_API_KEY'] = '2000a87a07a37c799c731975686e15079a8c188d'


In [ ]:
%cd drive/My\ Drive/Project/

/content/drive/My Drive/Project


In [ ]:
# Setup detectron2 logger
import detectron2
# from detectron2.utils.logger import setup_logger
# setup_logger()

# Import common libraries
import os
import json
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# Import detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer  # Look into moving to a custom training loop to add weights & biases
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.config import get_cfg

In [ ]:
# TOGGLE PROGRAM FUNCTIONALITY
TEST_INPUT = False  # Toggles image read test
CONFIG = "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml" # Pre-trained model path
TRAINING_CURVES = False  # Toggles Tensorboard training curves
EVALUATION = False  # Toggles model evaluation
WANDB_PROJ = 'cereal_trials'

# PROGRAM CONSTANTS


# FILE PATHS


# TRAINING PARAMETERS


In [ ]:
# Define some constants
ROOT_DIR = "custom"

TRAIN_DATASET_NAME = ROOT_DIR + "_train"
TRAIN_ANNOTATIONS = ROOT_DIR + "/train/annotations.json"
TRAIN_DIR = ROOT_DIR + "/train"

VAL_DATASET_NAME = ROOT_DIR + "_val"
VAL_ANNOTATIONS = ROOT_DIR + "/val/annotations.json"
VAL_DIR = ROOT_DIR + "/val"

In [ ]:
try:
  register_coco_instances(TRAIN_DATASET_NAME, {}, TRAIN_ANNOTATIONS, TRAIN_DIR)
  register_coco_instances(VAL_DATASET_NAME, {}, VAL_ANNOTATIONS, VAL_DIR)
except (AssertionError):
  print('Dataset has aready been registered!')

train_metadata = MetadataCatalog.get(TRAIN_DATASET_NAME)
dataset_dicts = DatasetCatalog.get(TRAIN_DATASET_NAME)


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



In [ ]:
if TEST_INPUT:
  for d in random.sample(dataset_dicts, 3):
      img = cv2.imread(d["file_name"])
      visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
      out = visualizer.draw_dataset_dict(d)
      cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
import logging
import os
from shutil import copyfile
from collections import OrderedDict
import torch
from torch.nn.parallel import DistributedDataParallel

import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer
from detectron2.config import get_cfg
from detectron2.data import (
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.engine import default_argument_parser, default_setup, launch
from detectron2.evaluation import (
    # CityscapesInstanceEvaluator,
    # CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    # LVISEvaluator,
    # PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    inference_on_dataset,
    print_csv_format,
)
from detectron2.modeling import build_model
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.utils.events import (
    CommonMetricPrinter,
    EventStorage,
    JSONWriter,
    TensorboardXWriter,
)

# Setup logger
logger = logging.getLogger("detectron2")

In [ ]:
def get_evaluator(cfg, dataset_name, output_folder=None):
    """
    Create evaluator(s) for a given dataset.
    This uses the special metadata "evaluator_type" associated with each builtin dataset.
    For your own dataset, you can simply create an evaluator manually in your
    script and do not have to worry about the hacky if-else logic here.

    # TODO: Edit docstring

    """
    if output_folder is None:
        output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
    evaluator_list = []

    evaluator_type = MetadataCatalog.get(dataset_name).evaluator_type
    if evaluator_type in ["sem_seg", "coco_panoptic_seg"]:
        evaluator_list.append(
            SemSegEvaluator(
                dataset_name,
                distributed=True,
                num_classes=cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES,
                ignore_label=cfg.MODEL.SEM_SEG_HEAD.IGNORE_VALUE,
                output_dir=output_folder,
            )
        )
    if evaluator_type in ["coco", "coco_panoptic_seg"]:
        evaluator_list.append(COCOEvaluator(dataset_name, cfg, True, output_folder))
    if evaluator_type == "coco_panoptic_seg":
        evaluator_list.append(COCOPanopticEvaluator(dataset_name, output_folder))
    # if evaluator_type == "cityscapes_instance":
    #     assert (
    #         torch.cuda.device_count() >= comm.get_rank()
    #     ), "CityscapesEvaluator currently do not work with multiple machines."
    #     return CityscapesInstanceEvaluator(dataset_name)
    # if evaluator_type == "cityscapes_sem_seg":
    #     assert (
    #         torch.cuda.device_count() >= comm.get_rank()
    #     ), "CityscapesEvaluator currently do not work with multiple machines."
    #     return CityscapesSemSegEvaluator(dataset_name)
    # if evaluator_type == "pascal_voc":
    #     return PascalVOCDetectionEvaluator(dataset_name)
    # if evaluator_type == "lvis":
    #     return LVISEvaluator(dataset_name, cfg, True, output_folder)
    if len(evaluator_list) == 0:
        raise NotImplementedError(
            "no Evaluator for the dataset {} with the type {}".format(dataset_name, evaluator_type)
        )
    if len(evaluator_list) == 1:
        return evaluator_list[0]
    return DatasetEvaluators(evaluator_list)

In [ ]:
def do_test(cfg, model):
    """

    # TODO: Write docstring
    """
    # Initialise results dictionary
    results = OrderedDict()

    # Loop through the datasets in the config file
    for dataset_name in cfg.DATASETS.TEST:
        data_loader = build_detection_test_loader(cfg, dataset_name)

        # Generate the evaluator
        evaluator = get_evaluator(
            cfg,
            dataset_name,
            os.path.join(cfg.OUTPUT_DIR, "inference", dataset_name)
        )

        # Run inference and add to results dictionary
        results_i = inference_on_dataset(model, data_loader, evaluator)
        results[dataset_name] = results_i
        
        logger.debug('Long the eval results on Weights & Biases')
        wandb.log(list(results_i.items())[0][1])

        if comm.is_main_process():
            logger.info("Evaluation results for {} in csv format:".format(dataset_name))
            print_csv_format(results_i)
    if len(results) == 1:
        results = list(results.values())[0]

    print(results)
    return results

In [ ]:
def do_train(cfg, model, resume=False):
    """

    # TODO: Write docstring
    """
    # Set the model to train
    model.train()

    # Create torch optimiser & schedulars
    optimizer = build_optimizer(cfg, model)
    scheduler = build_lr_scheduler(cfg, optimizer)

    # Create a torch checkpointer
    checkpointer = DetectionCheckpointer(
        model, cfg.OUTPUT_DIR, optimizer=optimizer, scheduler=scheduler
    )

    # Create starting checkpoint i.e. pre-trained model using weights from config
    start_iter = (
        checkpointer.resume_or_load(cfg.MODEL.WEIGHTS, resume=resume).get("iteration", -1) + 1
    )

    # Define the number of iterations
    max_iter = cfg.SOLVER.MAX_ITER

    # Create a periodic checkpointer at the configured period
    periodic_checkpointer = PeriodicCheckpointer(
        checkpointer, cfg.SOLVER.CHECKPOINT_PERIOD, max_iter=max_iter
    )

    # Export checkpoint data to terminal, JSON & tensorboard files
    writers = (
        [
            CommonMetricPrinter(max_iter),
            JSONWriter(os.path.join(cfg.OUTPUT_DIR, "metrics.json")),
            TensorboardXWriter(cfg.OUTPUT_DIR),
        ]
        if comm.is_main_process()
        else []
    )

    # Create a data loader to supply the model with training data
    data_loader = build_detection_train_loader(cfg)

    logger.info("Starting training from iteration {}".format(start_iter))
    with EventStorage(start_iter) as storage:
        for data, iteration in zip(data_loader, range(start_iter, max_iter)):
            iteration = iteration + 1
            storage.step()

            loss_dict = model(data)
            losses = sum(loss_dict.values())
            assert torch.isfinite(losses).all(), loss_dict

            loss_dict_reduced = {k: v.item() for k, v in comm.reduce_dict(loss_dict).items()}
            losses_reduced = sum(loss for loss in loss_dict_reduced.values())
            if comm.is_main_process():
                storage.put_scalars(total_loss=losses_reduced, **loss_dict_reduced)
          
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            storage.put_scalar("lr", optimizer.param_groups[0]["lr"], smoothing_hint=False)
            scheduler.step()

            # If eval period has been set, run test at defined interval
            if (
                cfg.TEST.EVAL_PERIOD > 0
                and iteration % cfg.TEST.EVAL_PERIOD == 0
                and iteration != max_iter
            ):
                do_test(cfg, model)
                comm.synchronize()

            if iteration - start_iter > 5 and (iteration % 20 == 0 or iteration == max_iter):
                logger.debug('Logging iteration and loss to Weights & Biases')
                wandb.log({"Iteration": iteration})
                wandb.log({"Loss": losses})

                for writer in writers:
                    writer.write()
            periodic_checkpointer.step(iteration)

In [ ]:
def setup(args):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)

    cfg.freeze()

    # Log the configuration file to OUTPUT_DIR
    default_setup(
        cfg, args
    )

    # Set up the weights and biases project
    logger.debug('Initialising Weights & Biases project')
    wandb.init(project=WANDB_PROJ, sync_tensorboard=False)

    # Load the yaml file and export it to wandb
    cfg_export = cfg.load_yaml_with_base(os.path.join(cfg.OUTPUT_DIR, "config.yaml"))
    logger.debug("Saving cfg file to Weights & Biases")
    wandb.config.update(cfg_export)

    return cfg

In [ ]:
def main(args):
    # Initialise the configuration datastructure
    cfg = setup(args)

    # Build a model from the configuration file
    model = build_model(cfg)

    logger.info("Model:\n{}".format(model))

    # If the 'eval_only' argument is present, load the last checkpoint
    # and return the results of the test function
    if args.eval_only:
        DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
            cfg.MODEL.WEIGHTS, resume=args.resume
        )
        return do_test(cfg, model)

    # Run the training loop
    do_train(cfg, model, resume=args.resume)

    logger.debug('Saving model to Weights & Biases')
    copyfile('custom/output/model_final.pth', wandb.run.dir + '/model_final.pth')

    # Return the results of the model test function
    return do_test(cfg, model)

In [ ]:
# Set up an argument string to pass into the main function
# workers = 2
# ims_per_batch = 2
# lr = 0.00025
# max_iter = 5
# seed = 27
# num_classes = 2
# config = model_zoo.get_config_file(CONFIG)
# weights = model_zoo.get_checkpoint_url(CONFIG)
# train = TRAIN_DATASET_NAME
# val = VAL_DATASET_NAME
# output = ROOT_DIR + '/output'
# eval_period = 100

# arg_string = f"--config-file {config} \
#               MODEL.WEIGHTS {weights} \
#               OUTPUT_DIR {output} \
#               DATASETS.TRAIN ('{train}',) \
#               DATASETS.TEST ('{val}',) \
#               DATALOADER.NUM_WORKERS {workers} \
#               SOLVER.IMS_PER_BATCH {ims_per_batch} \
#               SOLVER.BASE_LR {lr} \
#               SOLVER.MAX_ITER {max_iter} \
#               SEED {seed} \
#               MODEL.ROI_HEADS.NUM_CLASSES {num_classes} \
#               TEST.EVAL_PERIOD {eval_period}".split()

workers = 2
ims_per_batch = 2
lr = 0.001
warmup_iters = 1000
max_iter = 1500
step_low = 1000
step_high = 1500
gamma = 0.05
# seed = 27
num_classes = 2
config = model_zoo.get_config_file(CONFIG)
weights = model_zoo.get_checkpoint_url(CONFIG)
train = TRAIN_DATASET_NAME
val = VAL_DATASET_NAME
output = ROOT_DIR + '/output'
eval_period = 100

arg_string = f"--config-file {config} \
              MODEL.WEIGHTS {weights} \
              OUTPUT_DIR {output} \
              DATASETS.TRAIN ('{train}',) \
              DATASETS.TEST ('{val}',) \
              DATALOADER.NUM_WORKERS {workers} \
              SOLVER.IMS_PER_BATCH {ims_per_batch} \
              SOLVER.BASE_LR {lr} \
              SOLVER.WARMUP_ITERS {warmup_iters} \
              SOLVER.MAX_ITER {max_iter} \
              SOLVER.STEPS ({step_low},{step_high}) \
              SOLVER.GAMMA {gamma} \
              MODEL.ROI_HEADS.NUM_CLASSES {num_classes} \
              TEST.EVAL_PERIOD {eval_period}".split()

In [ ]:
  parser = default_argument_parser()
  args = parser.parse_args(arg_string)

In [ ]:
main(args)

[07/09 14:18:31 detectron2]: Rank of current process: 0. World size: 1
[07/09 14:18:33 detectron2]: Environment info:
------------------------  ---------------------------------------------------------------
sys.platform              linux
Python                    3.6.9 (default, Apr 18 2020, 01:56:04) [GCC 8.4.0]
numpy                     1.18.5
detectron2                0.1.3 @/usr/local/lib/python3.6/dist-packages/detectron2
detectron2 compiler       GCC 7.3
detectron2 CUDA compiler  10.1
detectron2 arch flags     sm_35, sm_37, sm_50, sm_52, sm_60, sm_61, sm_70, sm_75
DETECTRON2_ENV_MODULE     <not set>
PyTorch                   1.5.0+cu101 @/usr/local/lib/python3.6/dist-packages/torch
PyTorch debug build       False
CUDA available            True
GPU 0                     Tesla K80
CUDA_HOME                 /usr/local/cuda
NVCC                      Cuda compilation tools, release 10.1, V10.1.243
Pillow                    7.0.0
torchvision               0.6.0+cu101 @/usr/local/lib/

[07/09 14:18:35 detectron2]: Saving cfg file to Weights & Biases
[07/09 14:18:40 detectron2]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d

model_final_2d9806.pkl: 431MB [00:38, 11.3MB/s]                           

[07/09 14:19:18 fvcore.common.download]: Successfully downloaded /root/.torch/fvcore_cache/detectron2/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x/139653917/model_final_2d9806.pkl. 431414189 bytes.
[07/09 14:19:18 fvcore.common.file_io]: URL https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x/139653917/model_final_2d9806.pkl cached in /root/.torch/fvcore_cache/detectron2/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x/139653917/model_final_2d9806.pkl


[07/09 14:19:19 fvcore.common.checkpoint]: Reading a file from 'Detectron2 Model Zoo'
WARNING [07/09 14:19:19 fvcore.common.checkpoint]: Unable to load 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (3, 1024) in the model!
WARNING [07/09 14:19:19 fvcore.common.checkpoint]: Unable to load 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (3,) in the model!
WARNING [07/09 14:19:19 fvcore.common.checkpoint]: Unable to load 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (8, 1024) in the model!
WARNING [07/09 14:19:19 fvcore.common.checkpoint]: Unable to load 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (8,) in the model!
WARNING [07/09 14:19:19 fvcore.common.checkpoint]: Unable to load 'roi_heads.mask_head.predictor.weight' to 

In [ ]:
if TRAINING_CURVES:
  %load_ext tensorboard
  %tensorboard --logdir output

In [ ]:
if EVALUATION:
  from detectron2.evaluation import COCOEvaluator, inference_on_dataset
  from detectron2.data import build_detection_test_loader

  evaluator = COCOEvaluator(VAL_DATASET_NAME, cfg, False, output_dir= ROOT_DIR + "/output/")
  val_loader = build_detection_test_loader(cfg, VAL_DATASET_NAME)
  inference_on_dataset(trainer.model, val_loader, evaluator)